In [108]:
from copy import deepcopy
from IPython.display import clear_output
import time
from tqdm import tqdm
from itertools import permutations
import random

In [109]:
class Grid:
    def __init__(self, width, height, initial=None):
        self.width = width
        self.height = height
        
        if initial != None:
            self.grid = initial
        else:
            self.grid = []
            
            for i in range(height):
                self.grid.append([None] * self.width)
        
    # row returns the numbers in a row.
    # numbering starts at 0 from the top and moves downwards
    def row(self, row_num):
        return self.grid[row_num]
    
    # col returns the numbers in a column.
    # numbering starts at 0 on the left and moves right
    def col(self, col_num):
        return [self.grid[i][col_num] for i in range(self.height)]
    
    # diag1 returns the first diagonal going across the grid.
    # This means:
    # 1 | O | O
    # O | 2 | O
    # O | O | 3
    # Becomes:
    # [1, 2, 3]
    def diag1(self):
        if self.width != self.height:
            raise DimensionErr
            
        return [self.grid[i][i] for i in range(self.height)]
    
    # diag2 returns the second diagonal going across the grid.
    # This means:
    # O | O | 3
    # O | 2 | O
    # 1 | O | O
    # Becomes:
    # [1, 2, 3]
    def diag2(self):
        if self.width != self.height:
            raise DimensionErr
            
        return [self.grid[self.height - i - 1][i] for i in range(self.height)]
    
    # update updates the cell at (x, y), starting from the top (y) left (x).
    def update(self, x, y, val):
        self.grid[y][x] = val
        
    # first_none returns the coordinate of the first None filled cell, moving left to right, top to bottom.
    def first_none(self):
        for y in range(self.height):
            for x in range(self.width):
                if self.grid[y][x] == None:
                    return (x, y)
                
        return None
    
    # random_none returns the coordinate of a random cell containing None.
    def random_none(self):
        options = []
        
        for y in range(self.height):
            for x in range(self.width):
                if self.grid[y][x] == None:
                    options.append((x, y))
                    
        return random.choice(options)
    
    # magic returns true if the columns, rows and diagonals all add up to the same
    def magic(self, should_equal):
        for i in range(self.height):
            if sum(self.row(i)) != should_equal:
                return False
            
        for i in range(self.width):
            if sum(self.col(i)) != should_equal:
                return False

        return sum(self.diag1()) == sum(self.diag2()) == should_equal
    
    # partial_magic returns true if the columns, rows and diagonals that don't include None all add up to the same
    def partial_magic(self, should_equal):
        for i in range(self.height):
            row = self.row(i)
            if None not in row and sum(row) != should_equal:
                return False
            
        for i in range(self.width):
            col = self.col(i)
            if None not in col and sum(col) != should_equal:
                return False
            
        diag1 = self.diag1()
        if None not in diag1 and sum(diag1) != should_equal:
            return False
        
        diag2 = self.diag2()
        if None not in diag2 and sum(diag2) != should_equal:
            return False
        
        return True
    
    def __str__(self):
        str_rows = []
        
        for row in self.grid:
            str_rows.append(" | ".join(["_" if x == None else str(x) for x in row]))
            
        return ("\n" + "-" * len(str_rows[0]) + "\n").join(str_rows)
    
    def __repr__(self): return self.__str__()

In [110]:
# solve solves the diagonally-complete Latin square problem using a backtracking algorithm
# Diagonally-complete Latin squares can't have duplicates in the rows, columns or diagonals.
def solve(grid, inventory, magic):
    first_none = grid.first_none()
    
    if first_none == None:
        if grid.magic(magic):
            return grid
        else:
            return None
    
    x, y = first_none
    candidates = [num for num in inventory.keys() if inventory[num] > 0]
    
    for candidate in sorted(candidates):
        copied_grid = deepcopy(grid)
        copied_grid.grid[y][x] =  candidate
        
        # if adding this candidate means that a row/column/diagonal is not magic, then why try it?
        # if we have duplicates in the row or column we can skip it
        if not grid.partial_magic(magic) or candidate in grid.row(y) or candidate in grid.col(x):
            continue
            
        # prevent any duplicates in the diagonals
        if y == x and candidate in grid.diag1():
            continue
            
        if grid.height - y - 1 == x and candidate in grid.diag2():
            continue
        
        copied_inventory = deepcopy(inventory)
        copied_inventory[candidate] -= 1

        solved = solve(copied_grid, copied_inventory, magic)
        
        if solved != None:
            return solved

In [111]:
# general solves the general strong nxn square problem
def general(n):
    grid = Grid(n, n)
    return solve(grid, {i: n for i in range(n)}, n*(n-1)//2)

In [112]:
def complete(grid):
    nums = [num for row in grid for num in row]
    
    width = len(grid[0])
    height = len(grid)
    
    inventory = {i: width-nums.count(i) for i in range(width)}
    
    solved = solve(Grid(width, height, grid), inventory, sum(range(width)))
    
    return "No solutions" if solved == None else solved

In [113]:
def factorial(x):
    if x == 1:
        return 1
    
    else:
        return x * factorial(x-1)

In [37]:
def allowed_columns(n):
    valid = []
    total = 0

    for perm in tqdm(permutations(range(1, n)), total = factorial(n-1)):
        grid = [list(range(n))]
        
        for num in perm:
            grid.append([num] + [None for _ in range(n-1)])
            
        result = complete(grid)
        if result != "No solutions":
            valid.append((0, *perm))

        total += 1

    print(f"{len(valid)}/{total}")
    return valid

In [38]:
allowed_columns(4)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 625.52it/s]

2/6


[(0, 2, 3, 1), (0, 3, 1, 2)]

In [39]:
allowed_columns(5)

100%|███████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 139.64it/s]

8/24


[(0, 1, 3, 4, 2),
 (0, 1, 4, 2, 3),
 (0, 2, 4, 1, 3),
 (0, 2, 4, 3, 1),
 (0, 3, 1, 4, 2),
 (0, 3, 4, 2, 1),
 (0, 4, 1, 3, 2),
 (0, 4, 3, 1, 2)]

In [40]:
allowed_columns(6)

100%|██████████████████████████████████████████████████████████████| 120/120 [00:37<00:00,  3.16it/s]

32/120


[(0, 1, 2, 4, 5, 3),
 (0, 1, 2, 5, 3, 4),
 (0, 1, 3, 4, 5, 2),
 (0, 1, 3, 5, 2, 4),
 (0, 1, 4, 2, 5, 3),
 (0, 1, 4, 3, 5, 2),
 (0, 1, 5, 2, 3, 4),
 (0, 1, 5, 3, 2, 4),
 (0, 2, 3, 5, 1, 4),
 (0, 2, 3, 5, 4, 1),
 (0, 2, 5, 3, 1, 4),
 (0, 2, 5, 3, 4, 1),
 (0, 3, 2, 5, 1, 4),
 (0, 3, 2, 5, 4, 1),
 (0, 3, 5, 2, 1, 4),
 (0, 3, 5, 2, 4, 1),
 (0, 4, 1, 2, 5, 3),
 (0, 4, 1, 3, 5, 2),
 (0, 4, 2, 1, 5, 3),
 (0, 4, 2, 5, 3, 1),
 (0, 4, 3, 1, 5, 2),
 (0, 4, 3, 5, 2, 1),
 (0, 4, 5, 2, 3, 1),
 (0, 4, 5, 3, 2, 1),
 (0, 5, 1, 2, 4, 3),
 (0, 5, 1, 3, 4, 2),
 (0, 5, 2, 1, 4, 3),
 (0, 5, 2, 4, 1, 3),
 (0, 5, 3, 1, 4, 2),
 (0, 5, 3, 4, 1, 2),
 (0, 5, 4, 2, 1, 3),
 (0, 5, 4, 3, 1, 2)]

In [33]:
grid = Grid(6, 6)
grid.grid[0] = range(6)

%time complete(grid.grid)

CPU times: user 295 ms, sys: 2.89 ms, total: 298 ms
Wall time: 297 ms


0 | 1 | 2 | 3 | 4 | 5
---------------------
1 | 2 | 0 | 5 | 3 | 4
---------------------
4 | 3 | 5 | 0 | 2 | 1
---------------------
3 | 0 | 1 | 4 | 5 | 2
---------------------
5 | 4 | 3 | 2 | 1 | 0
---------------------
2 | 5 | 4 | 1 | 0 | 3